In [ ]:
import matplotlib.pyplot as plt
import os
from pathlib import Path
import numpy as np
from PIL import Image
import pandas as pd
import openpyxl
import re

## Clean the annotation files

In [ ]:
root_pth = Path(r"C:\Users\au761367\OneDrive - Aarhus universitet\Documents\projects\onestop\CamAlien annotations")
csv3_1_tth_pth = root_pth / "camalien-annotation_v3_1_TTH_GM2.xlsx"
csv3_1_pth = root_pth / "camalien-annotation_v3_1.xlsx"
csv1_pth=root_pth / "camalien-annotation_v1.xlsx"
csv2_pth=root_pth / "camalien-annotation_v2.xlsx"
csv3_pth=root_pth / "camalien-annotation_v3.xlsx"
all_csv_pth = [
    (csv1_pth, "Annotations"),
    (csv2_pth, "Sheet1"),
    (csv3_pth, "Sheet1"),
    (csv3_1_tth_pth, "Ark1"),
]

In [ ]:
def extract_and_clean_hyperlink_urls(excel_file, sheet_name, column_name):
    """
    Extracts the URL from an '=HYPERLINK(...)' formula, handling the Excel
    concatenation operator ('&') by removing it and combining the URL parts.
    """
    
    # 1. Load the DataFrame and Workbook
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    wb = openpyxl.load_workbook(excel_file, data_only=False)
    ws = wb[sheet_name]
    
    # 2. Find the 1-based column index
    column_index = -1
    for col_idx, cell in enumerate(ws[1]): 
        if cell.value == column_name:
            column_index = col_idx + 1 
            break
    
    if column_index == -1:
        print(f"Error: Column '{column_name}' not found.")
        return df

    # 3. Extract and Clean URLs
    urls = []
    
    for row_idx in range(2, len(df) + 2): 
        cell = ws.cell(row=row_idx, column=column_index)
        url_target = None
        
        # Check if the cell contains a formula
        if cell.data_type == 'f' and isinstance(cell.value, str):
            # formula = cell.value.upper()
            formula = cell.value
            
            if formula.startswith('=HYPERLINK('):
                # We need to target the ENTIRE first argument of the HYPERLINK function.
                # Find the content between '=HYPERLINK(' and the first separator (';' or ',')
                
                # Regex to capture the full first argument (the URL expression)
                # It looks for content inside HYPERLINK(...) up to the first ';' or ','
                # (Formula separator differs by regional Excel settings)
                match = re.search(r'=HYPERLINK\((.*?)[;,]', formula)
                
                if match:
                    # The full expression, e.g., '"...part1..."&"...part2..."'
                    url_expression = match.group(1).strip()
                    
                    # Clean the expression:
                    # 1. Remove all quotes (")
                    # 2. Remove all concatenation operators (&)
                    # This effectively stitches the string parts together.
                    cleaned_url = url_expression.replace('"', '').replace('&', '')
                    
                    # Ensure we have a URL remaining
                    if cleaned_url:
                        url_target = cleaned_url.strip()

        urls.append(url_target) 

    # 4. Add the extracted URLs as a new column
    # df[f'{column_name}_Cleaned_URL'] = urls
    df['imagelink'] = urls
    
    return df

def clean_hyperlink_and_save(excel_file, sheet_name, column_name, output_pth=None):
    df=extract_and_clean_hyperlink_urls(excel_file, sheet_name, column_name)
    if output_pth is None:
        output_pth = excel_file.with_name(excel_file.stem + '_cleanurl' + excel_file.suffix)
    df.to_excel(output_pth, index=False)

In [ ]:
# df=extract_and_clean_hyperlink_urls(csv1_pth, "Annotations", "imagelink")
df=extract_and_clean_hyperlink_urls(csv2_pth, "Sheet1", "imagelink")

In [ ]:
df.head(10)

In [ ]:
df['imagelink'].iloc[0]

In [ ]:
# df.to_excel(csv1_pth.with_name(csv1_pth.stem + "_clean" + csv1_pth.suffix), index=False)
df.to_excel(csv1_pth.with_name(csv2_pth.stem + "_clean" + csv2_pth.suffix), index=False)

In [ ]:
df.head()

In [ ]:
for c,s in all_csv_pth:
    clean_hyperlink_and_save(c, s, "imagelink")

## Compile/unify annotation files

In [ ]:
root_pth = Path(r"C:\Users\au761367\OneDrive - Aarhus universitet\Documents\projects\onestop\CamAlien annotations")
v3_1_tth_pth = root_pth / "camalien-annotation_v3_1_TTH_GM2.xlsx"
v3_1_pth = root_pth / "camalien-annotation_v3_1.xlsx"
v1_pth=root_pth / "camalien-annotation_v1_cleanurl.xlsx"
v2_pth=root_pth / "camalien-annotation_v2_cleanurl.xlsx"
v3_pth=root_pth / "camalien-annotation_v3_cleanurl.xlsx"
v31_pth=root_pth / "camalien-annotation_v3_1_TTH_GM2_cleanurl.xlsx"

In [ ]:
v31t = pd.read_excel(v3_1_tth_pth)
v31 = pd.read_excel(v3_1_pth)
v1 = pd.read_excel(v1_pth)
v2 = pd.read_excel(v2_pth)
v3 = pd.read_excel(v3_pth)
v31c = pd.read_excel(v31_pth)

In [ ]:
len(v31t), len(v31)

In [ ]:
v31tid=set(v31t['imagedataid'])
v31id=set(v31['imagedataid'])
len(v31id), len(v31tid)

In [ ]:
v31tid.difference(v31id)

In [ ]:
len(v31tid.intersection(v31id))

In [ ]:
v31id.difference(v31tid)

In [ ]:
dups_v31 = v31[v31['imagedataid'].duplicated(keep=False)]
dups_v31t = v31t[v31t['imagedataid'].duplicated(keep=False)]

In [ ]:
dups_v31t.sort_values(by=['imagedataid'])

In [ ]:
v31_nodup = v31.drop_duplicates()
v31t_nodup = v31t.drop_duplicates()

In [ ]:
len(v31t_nodup), len(v31_nodup)

In [ ]:
dups_v31 = v31_nodup[v31_nodup['imagedataid'].duplicated(keep=False)]
dups_v31t = v31t_nodup[v31t_nodup['imagedataid'].duplicated(keep=False)]
dups_v31t.sort_values(by=['imagedataid']).head(30)

In [ ]:
# Conlusion of the above: v3_1_tth contains more annotations, keep this one
# Fuse all csv/excel
v1.head()

In [ ]:
v3.head()

In [ ]:
v31c.head()

In [ ]:
v3.taxonname.unique()

In [ ]:
v31c.annotation.unique()